# Import package and data

In [49]:
# import the necessary libraries
import numpy as np 
import pandas as pd 
import os

# Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import pycountry
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import chart_studio.plotly as py
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
#py.init_notebook_mode(connected=True)

#Geographical Plotting
import folium
from folium import Choropleth, Circle, Marker
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

#Racing Bar Chart
import bar_chart_race as bcr
from IPython.display import HTML

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")# for pretty graphs

# Import Data

In [50]:
city_day = pd.read_csv('aqi_daily_1980_to_2021.csv')
print(city_day['State Name'].unique())
city_day.head()

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Country Of Mexico' 'Florida' 'Georgia' 'Hawaii' 'Idaho'
 'Illinois' 'Indiana' 'Iowa' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode Island' 'South Carolina' 'Tennessee' 'Texas' 'Utah' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' 'Delaware'
 'District Of Columbia' 'Kansas' 'Puerto Rico' 'Vermont' 'Virgin Islands'
 'South Dakota']


,State Name,Date,AQI,Category,Defining Parameter,Latitude,Longitude,County Name
0,Alabama,2021-01-01,30,Good,Ozone,34.289001,-85.970065,DeKalb
1,Alabama,2021-01-02,27,Good,Ozone,34.289001,-85.970065,DeKalb
2,Alabama,2021-01-03,34,Good,Ozone,34.289001,-85.970065,DeKalb
3,Alabama,2021-01-04,36,Good,Ozone,34.289001,-85.970065,DeKalb
4,Alabama,2021-01-05,31,Good,Ozone,34.289001,-85.970065,DeKalb


# Convert to Date Time format and Data Availability

In [51]:
city_day['Date'] = pd.to_datetime(city_day['Date'])
city_day['year'] = city_day['Date'].dt.year
city_day = city_day[2018 < city_day['year']]
print(f"The available data is between {city_day['Date'].min()} and {city_day['Date'].max()}")
city_day.head()

The available data is between 2019-01-01 00:00:00 and 2021-05-06 00:00:00


,State Name,Date,AQI,Category,Defining Parameter,Latitude,Longitude,County Name,year
0,Alabama,2021-01-01,30,Good,Ozone,34.289001,-85.970065,DeKalb,2021
1,Alabama,2021-01-02,27,Good,Ozone,34.289001,-85.970065,DeKalb,2021
2,Alabama,2021-01-03,34,Good,Ozone,34.289001,-85.970065,DeKalb,2021
3,Alabama,2021-01-04,36,Good,Ozone,34.289001,-85.970065,DeKalb,2021
4,Alabama,2021-01-05,31,Good,Ozone,34.289001,-85.970065,DeKalb,2021


# Missing value

In [52]:
# Missing values
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

missing_values= missing_values_table(city_day)
missing_values.style.background_gradient(cmap='Reds')

Your selected dataframe has 9 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


# Cities in the dataset

In [53]:
cities = city_day['County Name'].value_counts()
print(f'Total number of cities in the dataset : {len(cities)}')
print(cities.index)

Total number of cities in the dataset : 803
Index(['Washington', 'Jefferson', 'Montgomery', 'Lake', 'Clark', 'Hamilton',
       'Monroe', 'Franklin', 'Madison', 'Jackson',
       ...
       'Hyde', 'Socorro', 'Powell', 'Stoddard', 'Roosevelt', 'St Croix',
       'Winchester City', 'Hopewell City', 'Guernsey', 'Pend Oreille'],
      dtype='object', length=803)


# Effect of Lockdown on AQI

## AQI for some of the major cities of California

In [64]:
city_day = pd.DataFrame(city_day, columns=['County Name', 'Date', 'AQI', 'Category', 'Defining Parameter'])
# city_day.head()
cityname = ['Los Angeles', 'Fresno', 'Oakland']
AQI = city_day[city_day['County Name'].isin(cityname) ]
AQI.head()


,County Name,Date,AQI,Category,Defining Parameter
2127,Fresno,2021-01-01,55,Moderate,PM2.5
2128,Fresno,2021-01-02,26,Good,Ozone
2129,Fresno,2021-01-03,26,Good,Ozone
2130,Fresno,2021-01-04,31,Good,Ozone
2131,Fresno,2021-01-05,20,Good,Ozone


In [68]:
AQI_pivot = AQI.pivot(index='Date', columns='County Name', values='AQI')
AQI_pivot.fillna(method='bfill',inplace=True)
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=3, cols=1,
    #specs=[[{}, {}],
          # [{"colspan": 6}, None]],
    subplot_titles=('Los Angeles', 'Fresno', 'Oakland'))

fig.add_trace(go.Bar(x=AQI_pivot.index, y=AQI_pivot['Los Angeles'],
                    marker=dict(color=AQI_pivot['Los Angeles'],coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x=AQI_pivot.index, y=AQI_pivot['Fresno'],
                    marker=dict(color=AQI_pivot['Fresno'], coloraxis="coloraxis")),
              2, 1)
fig.add_trace(go.Bar(x=AQI_pivot.index, y=AQI_pivot['Oakland'],
                    marker=dict(color=AQI_pivot['Oakland'], coloraxis="coloraxis")),
              3, 1)
fig.update_layout(coloraxis=dict(colorscale='Temps'),showlegend=False,title_text="AQI Levels")

fig.update_layout(plot_bgcolor='white')

fig.update_layout( width=800,height=1200,shapes=[
      dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-19', x1= '2020-03-19'
    )
])

The black vertical line shows the date on which the first phase of lockdown ame into effect in California.
The above graph shows the variation of various pollutant levels, from Jan 2019 onwards till date.
Apparantely, all the above Indian cities seem to be a high level of pollution levels.
Clearly, there appears to be a rapid decline after 19th March,2020 in all the cities under sonsideration.

## AQI before and after Lockdown

In [72]:
AQI_beforeLockdown = AQI_pivot['2019-01-01':'2020-03-18']
AQI_afterLockdown = AQI_pivot['2020-03-19':'2020-04-19']
print(AQI_beforeLockdown.mean())
print(AQI_afterLockdown.mean())

County Name
Fresno         68.704289
Los Angeles    78.663657
Oakland        38.943567
dtype: float64
County Name
Fresno         41.7500
Los Angeles    48.5000
Oakland        35.5625
dtype: float64
